In [10]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import zipfile
import tensorflow as tf
from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import cv2
i=0
cap = cv2.VideoCapture("/home/laraib/Videos/video3.mp4")
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("/home/laraib/PycharmProjects/untitled2/venv/models-master/research")
#sys.path.append("..")
from object_detection.utils import ops as utils_ops
if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')


In [11]:
import io
from google.cloud import vision_v1p3beta1 as vision
from datetime import datetime

# Setup google authen client key
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'client_key.json'

# Source path content all images
SOURCE_PATH = "/home/laraib/Music/test/numberplate/"

In [12]:
# This is needed to display the images.
%matplotlib inline

In [13]:
sys.path.append("/home/laraib/PycharmProjects/untitled2/venv/models-master/research/object_detection")
from utils import label_map_util

from utils import visualization_utils as vis_util

In [14]:
# What model to download.
MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_oid_v4_2018_12_12'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'oid_v4_label_map.pbtxt')

In [15]:
#opener = urllib.request.URLopener()
#opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  tar_file.extract(file, os.getcwd())
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

In [16]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [17]:
NUM_CLASSES = 601
label_map = label_map_util.load_labelmap("/home/laraib/PycharmProjects/untitled2/venv/models-master/research/object_detection/data/"+
                                         "oid_v4_label_map.pbtxt")
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [18]:
with detection_graph.as_default():  
  with tf.Session(graph=detection_graph) as sess:    
    while (cap.isOpened()):
        i=i+1
        ret, image_np = cap.read()
        gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        # Actual detection.
        (boxes, scores, classes, num_detections) = sess.run(
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
        
        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int64),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8)
     
        cv2.imshow('object detection', cv2.resize(image_np, (800,600)))
        if cv2.waitKey(100000)  & (0xFF == ord('q')):
            #cv2.destroyAllWindows()
            break

In [1]:
import numpy as np
import cv2

cap = cv2.VideoCapture("/home/laraib/PycharmProjects/untitled2/VID-20190214-WA0000.mp4")

while(cap.isOpened()):
    ret, frame = cap.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    cv2.imshow('frame', gray)
    if cv2.waitKey(0) & (0xFF == ord('q')):
        break
cap.release()
cv2.destroyAllWindows()
    

error: OpenCV(4.0.0) /io/opencv/modules/imgproc/src/color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
